# Working with Text Data

http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

The goal of this guide is to explore some of the main scikit-learn tools on a single practical task: analysing a collection of text documents (newsgroups posts) on twenty different topics.
In this section we will see how to:
load the file contents and the categories
extract feature vectors suitable for machine learning
train a linear model to perform categorization
use a grid search strategy to find a good configuration of both the feature extraction components and the classifier

In [2]:
categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',
    categories=categories, shuffle=True, random_state=42)

In [3]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [4]:
len(twenty_train.data)

2257

In [5]:
len(twenty_train.filenames)

2257

In [7]:
print("\n".join(twenty_train.data[0].split("\n")[:3]))

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton


In [8]:
print(twenty_train.target_names[twenty_train.target[0]])

comp.graphics


Supervised learning algorithms will require a category label for each document in the training set. In this case the category is the name of the newsgroup which also happens to be the name of the folder holding the individual documents.
For speed and space efficiency reasons scikit-learn loads the target attribute as an array of integers that corresponds to the index of the category name in the target_names list. The category integer id of each sample is stored in the target attribute:

In [9]:
twenty_train.target[:10]

array([1, 1, 3, 3, 3, 3, 3, 2, 2, 2], dtype=int64)

In [10]:
for t in twenty_train.target[:10]:
    print(twenty_train.target_names[t])

comp.graphics
comp.graphics
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
sci.med
sci.med
sci.med


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(2257, 35788)

In [16]:
print(X_train_counts[0])

  (0, 230)	1
  (0, 12541)	1
  (0, 3166)	1
  (0, 14085)	1
  (0, 20459)	1
  (0, 35416)	1
  (0, 3062)	1
  (0, 2326)	2
  (0, 177)	2
  (0, 31915)	1
  (0, 33572)	1
  (0, 9338)	1
  (0, 26175)	1
  (0, 4378)	1
  (0, 17556)	1
  (0, 32135)	1
  (0, 15837)	1
  (0, 9932)	1
  (0, 32270)	1
  (0, 18474)	1
  (0, 27836)	1
  (0, 5195)	1
  (0, 12833)	2
  (0, 25337)	1
  (0, 25361)	1
  :	:
  (0, 5201)	1
  (0, 12051)	1
  (0, 587)	1
  (0, 20253)	1
  (0, 33597)	2
  (0, 32142)	5
  (0, 23915)	1
  (0, 16082)	1
  (0, 16881)	1
  (0, 25663)	1
  (0, 23122)	1
  (0, 17302)	2
  (0, 19780)	2
  (0, 16916)	2
  (0, 32493)	4
  (0, 17366)	1
  (0, 9805)	2
  (0, 31077)	1
  (0, 9031)	3
  (0, 21661)	3
  (0, 33256)	2
  (0, 4017)	2
  (0, 8696)	4
  (0, 29022)	1
  (0, 14887)	1


In [17]:
count_vect.vocabulary_.get(u'algorithm')

4690

From occurrences to frequencies

Occurrence count is a good start but there is an issue: longer documents will have higher average count values than shorter documents, even though they might talk about the same topics.
To avoid these potential discrepancies it suffices to divide the number of occurrences of each word in a document by the total number of words in the document: these new features are called tf for Term Frequencies.
Another refinement on top of tf is to downscale weights for words that occur in many documents in the corpus and are therefore less informative than those that occur only in a smaller portion of the corpus.
This downscaling is called tf–idf for “Term Frequency times Inverse Document Frequency”.
Both tf and tf–idf can be computed as follows:

In [18]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(2257, 35788)

In the above example-code, we firstly use the fit(..) method to fit our estimator to the data and secondly the transform(..) method to transform our count-matrix to a tf-idf representation. These two steps can be combined to achieve the same end result faster by skipping redundant processing. This is done through using the fit_transform(..) method as shown below, and as mentioned in the note in the previous section:

In [19]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2257, 35788)

Training a classifier

Now that we have our features, we can train a classifier to try to predict the category of a post. Let’s start with a naïve Bayes classifier, which provides a nice baseline for this task. scikit-learn includes several variants of this classifier; the one most suitable for word counts is the multinomial variant:

In [20]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

In [21]:
X_train_tfidf

<2257x35788 sparse matrix of type '<class 'numpy.float64'>'
	with 365886 stored elements in Compressed Sparse Row format>

In [23]:
print(X_train_tfidf[0])

  (0, 14887)	0.0167978060212
  (0, 29022)	0.13487105543
  (0, 8696)	0.314400065529
  (0, 4017)	0.124918175851
  (0, 33256)	0.118197024901
  (0, 21661)	0.196227989233
  (0, 9031)	0.384180393587
  (0, 31077)	0.0167978060212
  (0, 9805)	0.215672059147
  (0, 17366)	0.0744441018789
  (0, 32493)	0.0728377394162
  (0, 16916)	0.173584720477
  (0, 19780)	0.246455407094
  (0, 17302)	0.186260151092
  (0, 23122)	0.0363749163623
  (0, 25663)	0.0342907063629
  (0, 16881)	0.0360441471878
  (0, 16082)	0.113827386095
  (0, 23915)	0.0177623185636
  (0, 32142)	0.0886541625372
  (0, 33597)	0.0656757804319
  (0, 20253)	0.0168648929771
  (0, 587)	0.0596616201287
  (0, 12051)	0.037793189756
  (0, 5201)	0.0431619970071
  :	:
  (0, 25361)	0.119479381457
  (0, 25337)	0.0493588338398
  (0, 12833)	0.125601499991
  (0, 5195)	0.0310951485922
  (0, 27836)	0.0689905081067
  (0, 18474)	0.0199648817519
  (0, 32270)	0.0238711427382
  (0, 9932)	0.063505656472
  (0, 15837)	0.0541740417987
  (0, 32135)	0.0491023738045
  (0

o try to predict the outcome on a new document we need to extract the features using almost the same feature extracting chain as before. The difference is that we call transform instead of fit_transform on the transformers, since they have already been fit to the training set:

In [24]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


Building a pipeline

In order to make the vectorizer => transformer => classifier easier to work with, scikit-learn provides a Pipeline class that behaves like a compound classifier:

In [25]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
])

In [26]:
text_clf = text_clf.fit(twenty_train.data, twenty_train.target)

Evaluation of the performance on the test set

Evaluating the predictive accuracy of the model is equally easy:

In [27]:
import numpy as np
twenty_test = fetch_20newsgroups(subset='test',
    categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.83488681757656458

I.e., we achieved 83.4% accuracy. Let’s see if we can do better with a linear support vector machine (SVM), which is widely regarded as one of the best text classification algorithms (although it’s also a bit slower than naïve Bayes). We can change the learner by just plugging a different classifier object into our pipeline:

In [28]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, n_iter=5, random_state=42)),
])
_ = text_clf.fit(twenty_train.data, twenty_train.target)
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.9127829560585885

scikit-learn further provides utilities for more detailed performance analysis of the results:

In [29]:
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted,
    target_names=twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.95      0.81      0.87       319
         comp.graphics       0.88      0.97      0.92       389
               sci.med       0.94      0.90      0.92       396
soc.religion.christian       0.90      0.95      0.93       398

           avg / total       0.92      0.91      0.91      1502



In [30]:
metrics.confusion_matrix(twenty_test.target, predicted)

array([[258,  11,  15,  35],
       [  4, 379,   3,   3],
       [  5,  33, 355,   3],
       [  5,  10,   4, 379]])

As expected the confusion matrix shows that posts from the newsgroups on atheism and christian are more often confused for one another than with computer graphics.

Parameter tuning using grid search¶

We’ve already encountered some parameters such as use_idf in the TfidfTransformer. Classifiers tend to have many parameters as well; e.g., MultinomialNB includes a smoothing parameter alpha and SGDClassifier has a penalty parameter alpha and configurable loss and penalty terms in the objective function (see the module documentation, or use the Python help function, to get a description of these).
Instead of tweaking the parameters of the various components of the chain, it is possible to run an exhaustive search of the best parameters on a grid of possible values. We try out all classifiers on either words or bigrams, with or without idf, and with a penalty parameter of either 0.01 or 0.001 for the linear SVM:

In [31]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-2, 1e-3),
}

Obviously, such an exhaustive search can be expensive. If we have multiple CPU cores at our disposal, we can tell the grid searcher to try these eight parameter combinations in parallel with the n_jobs parameter. If we give this parameter a value of -1, grid search will detect how many cores are installed and uses them all:

In [32]:
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)

In [33]:
gs_clf = gs_clf.fit(twenty_train.data[:400], twenty_train.target[:400])

The result of calling fit on a GridSearchCV object is a classifier that we can use to predict:

In [34]:
twenty_train.target_names[gs_clf.predict(['God is love'])[0]]

'soc.religion.christian'

The object’s best_score_ and best_params_ attributes store the best mean score and the parameters setting corresponding to that score:

In [36]:
print(gs_clf.best_score_)

for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

0.9
clf__alpha: 0.001
tfidf__use_idf: True
vect__ngram_range: (1, 1)


A more detailed summary of the search is available at gs_clf.cv_results_.
The cv_results_ parameter can be easily imported into pandas as a DataFrame for further inspection.

In [39]:
gs_clf.cv_results_

{'mean_fit_time': array([ 0.20347699,  0.85127091,  0.21615386,  0.91898632,  0.23450033,
         1.00071128,  0.267857  ,  0.93799861]),
 'mean_score_time': array([ 0.06771541,  0.22382506,  0.07938925,  0.25918341,  0.08706204,
         0.30688334,  0.09373418,  0.21548637]),
 'mean_test_score': array([ 0.8775,  0.875 ,  0.765 ,  0.78  ,  0.9   ,  0.89  ,  0.7675,  0.81  ]),
 'mean_train_score': array([ 0.99374372,  1.        ,  0.94123886,  0.97623272,  1.        ,
         1.        ,  0.98499057,  1.        ]),
 'param_clf__alpha': masked_array(data = [0.01 0.01 0.01 0.01 0.001 0.001 0.001 0.001],
              mask = [False False False False False False False False],
        fill_value = ?),
 'param_tfidf__use_idf': masked_array(data = [True True False False True True False False],
              mask = [False False False False False False False False],
        fill_value = ?),
 'param_vect__ngram_range': masked_array(data = [(1, 1) (1, 2) (1, 1) (1, 2) (1, 1) (1, 2) (1, 1) (1, 2